In [1]:
import os
import numpy as np
import pandas as pd

Paths

In [2]:
RAW_DATA = os.path.join("..","data","raw")
PROCESSED_DATA = os.path.join("..","data","processed")

In [3]:
train_path = os.path.join(RAW_DATA, "train.csv")
test_path = os.path.join(RAW_DATA, "test.csv")

Load data

In [4]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [5]:
print("train shape:", df_train.shape)
print("test shape:", df_test.shape)

train shape: (1460, 81)
test shape: (1459, 80)


Check for missing values

In [6]:
missing = df_train.isnull().sum()
missing = missing[missing>0].sort_values(ascending = False)
missing_percent = (missing / len(df_train)) * 100
missing_summary = pd.DataFrame({"Missing values":missing, "Percent": missing_percent})
missing_summary

,Missing values,Percent
PoolQC,1453,99.520548
MiscFeature,1406,96.301370
Alley,1369,93.767123
Fence,1179,80.753425
MasVnrType,872,59.726027
FireplaceQu,690,47.260274
LotFrontage,259,17.739726
GarageType,81,5.547945
GarageYrBlt,81,5.547945
GarageFinish,81,5.547945
